FaceJ:
    a simplified YOLO-based face detection model trained with WIDER_FACE datasets;

YOLO training pipeline:
    adapted from MXNET/GLUON tutorial;

WIDER_FACE:
    converted from jpg to rec using gluon tool im2rec;

Architecture:
    1. Resnet18: high speed;
    2. Resnet34: high accuracy;
    3. originally designed; ideas adopted from https://towardsdatascience.com/faced-cpu-real-time-face-detection-using-deep-learning-1488681c1602

In [ ]:
import numpy as np
import mxnet as mx
import time
import datetime
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib as mpl

from mxboard import SummaryWriter
from mxnet import gluon
from mxnet import image
from mxnet import nd
from mxnet.gluon import nn
from mxnet.gluon import Block, HybridBlock
from mxnet.gluon.model_zoo import vision
from mxnet import metric
from mxnet import init
from mxnet import gpu
from mxnet import autograd
from mxnet.gluon import nn

%matplotlib inline

In [ ]:
def transform_center(xy):
    """Given x, y prediction after sigmoid(), convert to relative coordinates (0, 1) on image."""
    b, h, w, n, s = xy.shape
    offset_y = nd.tile(nd.arange(0, h, repeat=(w * n * 1), ctx=xy.context).reshape((1, h, w, n, 1)), (b, 1, 1, 1, 1))
    # print(offset_y[0].asnumpy()[:, :, 0, 0])
    offset_x = nd.tile(nd.arange(0, w, repeat=(n * 1), ctx=xy.context).reshape((1, 1, w, n, 1)), (b, h, 1, 1, 1))
    # print(offset_x[0].asnumpy()[:, :, 0, 0])
    x, y = xy.split(num_outputs=2, axis=-1)
    x = (x + offset_x) / w
    y = (y + offset_y) / h
    return x, y

def transform_size(wh, anchors):
    """Given w, h prediction after exp() and anchor sizes, convert to relative width/height (0, 1) on image"""
    b, h, w, n, s = wh.shape
    aw, ah = nd.tile(nd.array(anchors, ctx=wh.context).reshape((1, 1, 1, -1, 2)), (b, h, w, 1, 1)).split(num_outputs=2, axis=-1)
    w_pred, h_pred = nd.exp(wh).split(num_outputs=2, axis=-1)
    w_out = w_pred * aw / w
    h_out = h_pred * ah / h
    return w_out, h_out

def yolo2_forward(x, num_class, anchor_scales):
    """Transpose/reshape/organize convolution outputs."""
    stride = num_class + 5
    # transpose and reshape, 4th dim is the number of anchors
    x = x.transpose((0, 2, 3, 1))
    x = x.reshape((0, 0, 0, -1, stride))
    # now x is (batch, m, n, stride), stride = num_class + 1(object score) + 4(coordinates)
    # class probs
    cls_pred = x.slice_axis(begin=0, end=num_class, axis=-1)
    # object score
    score_pred = x.slice_axis(begin=num_class, end=num_class + 1, axis=-1)
    score = nd.sigmoid(score_pred)
    # center prediction, in range(0, 1) for each grid
    xy_pred = x.slice_axis(begin=num_class + 1, end=num_class + 3, axis=-1)
    xy = nd.sigmoid(xy_pred)
    # width/height prediction
    wh = x.slice_axis(begin=num_class + 3, end=num_class + 5, axis=-1)
    # convert x, y to positions relative to image
    x, y = transform_center(xy)
    # convert w, h to width/height relative to image
    w, h = transform_size(wh, anchor_scales)
    # cid is the argmax channel
    cid = nd.argmax(cls_pred, axis=-1, keepdims=True)
    # convert to corner format boxes
    half_w = w / 2
    half_h = h / 2
    left = nd.clip(x - half_w, 0, 1)
    top = nd.clip(y - half_h, 0, 1)
    right = nd.clip(x + half_w, 0, 1)
    bottom = nd.clip(y + half_h, 0, 1)
    output = nd.concat(*[cid, score, left, top, right, bottom], dim=4)
    return output, cls_pred, score, nd.concat(*[xy, wh], dim=4)

def corner2center(boxes, concat=True):
    """Convert left/top/right/bottom style boxes into x/y/w/h format"""
    left, top, right, bottom = boxes.split(axis=-1, num_outputs=4)
    x = (left + right) / 2
    y = (top + bottom) / 2
    width = right - left
    height = bottom - top
    if concat:
        last_dim = len(x.shape) - 1
        return nd.concat(*[x, y, width, height], dim=last_dim)
    return x, y, width, height

def center2corner(boxes, concat=True):
    """Convert x/y/w/h style boxes into left/top/right/bottom format"""
    x, y, w, h = boxes.split(axis=-1, num_outputs=4)
    w2 = w / 2
    h2 = h / 2
    left = x - w2
    top = y - h2
    right = x + w2
    bottom = y + h2
    if concat:
        last_dim = len(left.shape) - 1
        return nd.concat(*[left, top, right, bottom], dim=last_dim)
    return left, top, right, bottom

def yolo2_target(scores, boxes, labels, anchors, ignore_label=-1, thresh=0.5):
    """Generate training targets given predictions and labels."""
    b, h, w, n, _ = scores.shape
    anchors = np.reshape(np.array(anchors), (-1, 2))
    #scores = nd.slice_axis(outputs, begin=1, end=2, axis=-1)
    #boxes = nd.slice_axis(outputs, begin=2, end=6, axis=-1)
    gt_boxes = nd.slice_axis(labels, begin=1, end=5, axis=-1)
    target_score = nd.zeros((b, h, w, n, 1), ctx=scores.context)
    target_id = nd.ones_like(target_score, ctx=scores.context) * ignore_label
    target_box = nd.zeros((b, h, w, n, 4), ctx=scores.context)
    sample_weight = nd.zeros((b, h, w, n, 1), ctx=scores.context)
    for b in range(output.shape[0]):
        # find the best match for each ground-truth
        label = labels[b].asnumpy()
        valid_label = label[np.where(label[:, 0] > -0.5)[0], :]
        # shuffle because multi gt could possibly match to one anchor, we keep the last match randomly
        np.random.shuffle(valid_label)
        for l in valid_label:
            gx, gy, gw, gh = (l[1] + l[3]) / 2, (l[2] + l[4]) / 2, l[3] - l[1], l[4] - l[2]
            ind_x = int(gx * w)
            ind_y = int(gy * h)
            tx = gx * w - ind_x
            ty = gy * h - ind_y
            gw = gw * w
            gh = gh * h
            # find the best match using width and height only, assuming centers are identical
            intersect = np.minimum(anchors[:, 0], gw) * np.minimum(anchors[:, 1], gh)
            ovps = intersect / (gw * gh + anchors[:, 0] * anchors[:, 1] - intersect)
            best_match = int(np.argmax(ovps))
            target_id[b, ind_y, ind_x, best_match, :] = l[0]
            target_score[b, ind_y, ind_x, best_match, :] = 1.0
            tw = np.log(gw / anchors[best_match, 0])
            th = np.log(gh / anchors[best_match, 1])
            target_box[b, ind_y, ind_x, best_match, :] = mx.nd.array([tx, ty, tw, th])
            sample_weight[b, ind_y, ind_x, best_match, :] = 1.0
            # print('ind_y', ind_y, 'ind_x', ind_x, 'best_match', best_match, 't', tx, ty, tw, th, 'ovp', ovps[best_match], 'gt', gx, gy, gw/w, gh/h, 'anchor', anchors[best_match, 0], anchors[best_match, 1])
    return target_id, target_score, target_box, sample_weight

class YOLO2Output(HybridBlock):
    def __init__(self, num_class, anchor_scales, **kwargs):
        super(YOLO2Output, self).__init__(**kwargs)
        assert num_class > 0, "number of classes should > 0, given {}".format(num_class)
        self._num_class = num_class
        assert isinstance(anchor_scales, (list, tuple)), "list or tuple of anchor scales required"
        assert len(anchor_scales) > 0, "at least one anchor scale required"
        for anchor in anchor_scales:
            assert len(anchor) == 2, "expected each anchor scale to be (width, height), provided {}".format(anchor)
        self._anchor_scales = anchor_scales
        out_channels = len(anchor_scales) * (num_class + 1 + 4)
        with self.name_scope():
            self.output = nn.Conv2D(out_channels, 1, 1)

    def hybrid_forward(self, F, x, *args):
        return self.output(x)

def get_iterators(data_shape, batch_size):
    class_names = ['face', 'dummy']
    num_class = len(class_names)
    train_iter = image.ImageDetIter(
        batch_size=batch_size,
        data_shape=(3, data_shape, data_shape),
        path_imgrec=data_dir+'wider_train.rec',
        path_imgidx=data_dir+'wider_train.idx',
        shuffle=True,
        mean=True,
        std=True,
        rand_crop=1,
        min_object_covered=0.95,
        max_attempts=200)
    val_iter = image.ImageDetIter(
        batch_size=batch_size,
        data_shape=(3, data_shape, data_shape),
        path_imgrec=data_dir+'wider_val.rec',
        shuffle=False,
        mean=True,
        std=True)
    return train_iter, val_iter, class_names, num_class

class LossRecorder(mx.metric.EvalMetric):
    """LossRecorder is used to record raw loss so we can observe loss directly
    """
    def __init__(self, name):
        super(LossRecorder, self).__init__(name)

    def update(self, labels, preds=0):
        """Update metric with pure loss
        """
        for loss in labels:
            if isinstance(loss, mx.nd.NDArray):
                loss = loss.asnumpy()
            self.sum_metric += loss.sum()
            self.num_inst += 1

def process_image(fname):
    with open(fname, 'rb') as f:
        im = image.imdecode(f.read())
    # resize to data_shape
    data = image.imresize(im, data_shape, data_shape)
    # minus rgb mean, divide std
    data = (data.astype('float32') - rgb_mean) / rgb_std
    # convert to batch x channel x height xwidth
    return data.transpose((2,0,1)).expand_dims(axis=0), im

def predict(x):
    x = net(x)
    output, cls_prob, score, xywh = yolo2_forward(x, 2, scales)
    return nd.contrib.box_nms(output.reshape((0, -1, 6)))

def display(im, out, threshold=0.5):
    plt.imshow(im.asnumpy())
    for row in out:
        row = row.asnumpy()
        class_id, score = int(row[0]), row[1]
        if class_id < 0 or score < threshold:
            continue
        color = colors[class_id%len(colors)]
        box = row[2:6] * np.array([im.shape[1],im.shape[0]]*2)
        rect = box_to_rect(nd.array(box), color, 2)
        plt.gca().add_patch(rect)
        text = class_names[class_id]
        plt.gca().text(box[0], box[1],
                       '{:s} {:.2f}'.format(text, score),
                       bbox=dict(facecolor=color, alpha=0.5),
                       fontsize=10, color='white')
        plt.savefig('1image_mulifaces_bx.png')
    plt.show()
    
mpl.rcParams['figure.dpi']= 120
def box_to_rect(box, color, linewidth=3):
    """convert an anchor box to a matplotlib rectangle"""
    box = box.asnumpy()
    return plt.Rectangle(
        (box[0], box[1]), box[2]-box[0], box[3]-box[1],
        fill=False, edgecolor=color, linewidth=linewidth)

In [ ]:
# WIDER_FACE directory
data_dir = '/data/face_detector/wider_voc/train/'

# ImageDetIter WIDER_FACE
data_shape = 256
batch_size = 16
rgb_mean = nd.array([123, 117, 104])
rgb_std = nd.array([58.395, 57.12, 57.375])

train_data, test_data, class_names, num_class = get_iterators(
    data_shape, batch_size)

batch = train_data.next()
print(batch)

_, figs = plt.subplots(3, 3, figsize=(6,6))
for i in range(3):
    for j in range(3):
        img, labels = batch.data[0][3*i+j], batch.label[0][3*i+j]
        img = img.transpose((1, 2, 0)) * rgb_std + rgb_mean
        img = img.clip(0,255).asnumpy()/255
        fig = figs[i][j]
        fig.imshow(img)
        for label in labels:
            rect = box_to_rect(label[1:5]*data_shape,'red',2)
            fig.add_patch(rect)
        fig.axes.get_xaxis().set_visible(False)
        fig.axes.get_yaxis().set_visible(False)
plt.show()

In [ ]:
sce_loss = gluon.loss.SoftmaxCrossEntropyLoss(from_logits=False)
l1_loss = gluon.loss.L1Loss()

obj_loss = LossRecorder('objectness_loss')
cls_loss = LossRecorder('classification_loss')
box_loss = LossRecorder('box_refine_loss')

positive_weight = 5.0
negative_weight = 0.1
class_weight = 1.0
box_weight = 5.0

In [ ]:
net_ori = nn.HybridSequential()
class Residual(nn.HybridBlock):
    def __init__(self, channels, same_shape=True, **kwargs):
        super(Residual, self).__init__(**kwargs)
        self.same_shape = same_shape
        with self.name_scope():
            strides = 1 if same_shape else 2
            self.conv1 = nn.Conv2D(channels, kernel_size=3, padding=1,
                                  strides=strides)
            self.bn1 = nn.BatchNorm()
            self.conv2 = nn.Conv2D(channels, kernel_size=3, padding=1)
            self.bn2 = nn.BatchNorm()
            if not same_shape:
                self.conv3 = nn.Conv2D(channels, kernel_size=1,
                                      strides=strides)

    def hybrid_forward(self, F, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        if not self.same_shape:
            x = self.conv3(x)
        return F.relu(out + x)

conv1 = nn.Conv2D(channels=32, kernel_size=3, strides=1, padding=1)
bn1 = nn.BatchNorm()
relu = nn.Activation(activation='relu')
maxpool1 = nn.MaxPool2D()
net_ori.add(conv1)
net_ori.add(bn1)
net_ori.add(relu)
net_ori.add(maxpool1)
for _ in range(3):
    net_ori.add(Residual(channels=32))
net_ori.add(Residual(channels=64, same_shape=False))
for _ in range(2):
    net_ori.add(Residual(channels=64))

# anchor scales, try adjust it yourself
scales = [[3.3004, 3.59034],
          [9.84923, 8.23783]]

# use 2 classes, 1 as dummy class, otherwise softmax won't work
predictor = YOLO2Output(2, scales)
predictor.initialize()
net_ori.add(predictor)

In [ ]:
pretrained = vision.get_model('resnet18_v1', pretrained=True).features
net = nn.HybridSequential()
for i in range(len(pretrained) - 2):
    net.add(pretrained[i])
net

In [ ]:
pretrained = vision.get_model('resnet18_v1', pretrained=True).features
net = nn.HybridSequential()
for i in range(len(pretrained) - 2):
    net.add(pretrained[i])

# anchor scales, try adjust it yourself
scales = [[3.3004, 3.59034],
          [9.84923, 8.23783]]

# use 2 classes, 1 as dummy class, otherwise softmax won't work
predictor = YOLO2Output(2, scales)
predictor.initialize()
net.add(predictor)

In [ ]:
# training model
lr_period = 50
lr_decay = 0.1
ctx = mx.cpu()
net.collect_params().reset_ctx(ctx)
trainer = gluon.Trainer(net.collect_params(),
                        'sgd', {'learning_rate': 1, 'wd': 5e-4})

epoch = 0
train_data.reset()
cls_loss.reset()
obj_loss.reset()
box_loss.reset()
tic = time.time()
for i, batch in enumerate(train_data):
    x = batch.data[0].as_in_context(ctx)
    y = batch.label[0].as_in_context(ctx)
    #print('batch x ==>', x.shape)
    #print('batch y ==>', y)
    with autograd.record():
        x = net(x)
        #print(x)
        output, cls_pred, score, xywh = yolo2_forward(x, 2, scales)
        #print(xywh)
        with autograd.pause():
            tid, tscore, tbox, sample_weight = yolo2_target(score, xywh, y, scales, thresh=0.5)
        #print(class_weight)
        # losses
        loss1 = sce_loss(cls_pred, tid, sample_weight * class_weight)
        #print('loss1 predicted, truth ==>', loss1)
        score_weight = nd.where(sample_weight > 0,
                                    nd.ones_like(sample_weight) * positive_weight,
                                    nd.ones_like(sample_weight) * negative_weight)
        loss2 = l1_loss(score, tscore, score_weight)
        #print('loss2 predicted, truth ==>', score - tscore)
        loss3 = l1_loss(xywh, tbox, sample_weight * box_weight)
        print('loss3 predicted ==>', xywh)
        print('loss3 truth ==>', tbox)
        loss = loss1 + loss2 + loss3
    loss.backward()
    trainer.step(batch_size)
    # update metrics
    cls_loss.update(loss1)
    obj_loss.update(loss2)
    box_loss.update(loss3)

In [ ]:
net.hybridize()
unique_id = str(datetime.datetime.now()).replace(' ', '-').replace(':', '-').replace('.','-')
with SummaryWriter(logdir='/data/face_detector/logs_gluon/logs-'+unique_id, flush_secs=5) as sw:
    try:
        global_step = 0
        for epoch in range(1):
            # reset data iterators and metrics
            thefile = open('/data/face_detector/logs_gluon/logs-'+unique_id+'/faceJ'+'-'+str(epoch)+'.txt', 'w')
            if epoch > 0 and epoch % lr_period == 0:
                trainer.set_learning_rate(trainer.learning_rate * lr_decay)
            train_data.reset()
            cls_loss.reset()
            obj_loss.reset()
            box_loss.reset()
            tic = time.time()
            for i, batch in enumerate(train_data):
                global_step += 1
                x = batch.data[0].as_in_context(ctx)
                y = batch.label[0].as_in_context(ctx)
                with autograd.record():
                    x = net(x)
                    output, cls_pred, score, xywh = yolo2_forward(x, 2, scales)
                    with autograd.pause():
                        tid, tscore, tbox, sample_weight = yolo2_target(score, xywh, y, scales, thresh=0.5)
                    # losses
                    loss1 = sce_loss(cls_pred, tid, sample_weight * class_weight)
                    score_weight = nd.where(sample_weight > 0,
                                            nd.ones_like(sample_weight) * positive_weight,
                                            nd.ones_like(sample_weight) * negative_weight)
                    loss2 = l1_loss(score, tscore, score_weight)
                    loss3 = l1_loss(xywh, tbox, sample_weight * box_weight)
                    loss = loss1 + loss2 + loss3
                loss.backward()
                trainer.step(batch_size)
                # update metrics
                cls_loss.update(loss1)
                obj_loss.update(loss2)
                box_loss.update(loss3)

                if global_step%100 == 0:
                    sw.add_scalar(tag='cls_loss',value=cls_loss.get(), global_step=global_step)
                    sw.add_scalar(tag='obj_loss',value=obj_loss.get(), global_step=global_step)
                    sw.add_scalar(tag='box_loss',value=box_loss.get(), global_step=global_step)
                    sw.add_scalar(tag='epoch', value=epoch, global_step=global_step)

            now_lr = trainer.learning_rate
            print('Epoch %2d, train %s %.5f, %s %.5f, %s %.5f time %.1f sec, learning rate %.15f' % (
                epoch, *cls_loss.get(), *obj_loss.get(), *box_loss.get(), time.time()-tic, now_lr))
            thefile.write('Epoch %2d, train %s %.5f, %s %.5f, %s %.5f time %.1f sec' % (epoch, *cls_loss.get(), *obj_loss.get(), *box_loss.get(), time.time()-tic))
            thefile.close()
            #net.export('/data/face_detector/logs_gluon/logs-'+unique_id+'/FaceJparams-'+unique_id+str(epoch))
            sw.close()

    except KeyboardInterrupt:
        print("KeyboardInterrupted")
        sw.close()
        exit(0)
        pass

In [ ]:
#parameters have to be consistent with FaceJ.py

#resnet18 base model
#net.collect_params().load('/data/face_detector/logs_gluon/logs-2019-01-15-14-13-37-669618/FaceJparams-2019-01-15-14-13-37-66961810-0000.params',ctx=ctx, allow_missing=False, ignore_extra=True)

#resnet34 base model
net.collect_params().load('/data/face_detector/logs_gluon/logs-2019-01-16-08-50-54-544639/FaceJparams-2019-01-16-08-50-54-544639295-0000.params',ctx=ctx, allow_missing=False, ignore_extra=True)

In [ ]:
# predic
x, im = process_image('/data/face_detector/WIDER_test/9_Press_Conference_Press_Conference_9_783.jpg')
out = predict(x.as_in_context(ctx))
out.shape

In [ ]:
#plot predicted face with bounding box
mpl.rcParams['figure.figsize'] = (8,8)
colors = ['blue', 'green', 'red', 'black', 'magenta']
display(im, out[0], threshold=0.5)

In [ ]:
net.collect_params().load('/data/face_detector/logs_gluon/logs-2019-01-16-08-50-54-544639/FaceJparams-2019-01-16-08-50-54-544639295-0000.params',ctx=ctx, allow_missing=False, ignore_extra=True)
x, im = process_image('/data/face_detector/WIDER_test/9_Press_Conference_Press_Conference_9_783.jpg')
out = predict(x.as_in_context(ctx))
mpl.rcParams['figure.figsize'] = (8,8)
colors = ['blue', 'green', 'red', 'black', 'magenta']
display(im, out[0], threshold=0.5)